In [1]:
import numpy as np
import sys, os
cwd_split = os.getcwd().split(os.sep)
root = os.path.abspath("")
sys.path.insert(0, f"{root}/build/python")

import mitsuba as mi
# mi.set_variant("cuda_ad_rgb")
mi.set_variant("cuda_ad_rgb_double")
import drjit as dr

from gaussnewton_test import conjugate_gradient, AlbedoScene

In [2]:
a = mi.TensorXf([0.15, 1.0, 0.8, 0.0, 0.0, 0.0], [2,3])
b = dr.ravel(a)
print(len(b))

len_values = {"a": 1,
              "b": 2,
              "c": 3}
dr.sum(len_values.values())

6


6

In [3]:
dr.arange(mi.UInt32, 0, 10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [4]:

scene_dict = {
    "type": "scene",
    'integrator': {
        'type': 'direct',
        # 'max_depth': 2
    },
    'sensor': {
        'type': 'perspective',
        'fov': 30,
        'to_world': mi.ScalarTransform4f.look_at(
            origin=[0, 0, 1],
            target=[0, 0, 0],
            up=[0, 1, 0]
        ),
        'film': {
            'type': 'hdrfilm',
            'width': 32,
            'height': 32,
            'filter': { 'type': 'box' }
        }
    },
    'shape1': {
        'type': 'rectangle',
        'emitter': {
            'type': 'area',
            'radiance': {
                'type': 'rgb',
                'value': [0.1, 0.2, 0.3],
            }
        }
    }
}

scene = mi.load_dict(scene_dict)
params = mi.traverse(scene)
key = {'shape1.emitter.radiance.value'}

# Instantiate the scene manager
experiment_idx = 0
sc = AlbedoScene(scene, params, key, _EXP_IDX = experiment_idx)

# %%
RED = '\033[91m'
RESET = '\033[0m'
iteration_count = 20
image, loss = sc.eval_func()
for it in range(iteration_count):

    # Compute the Gauss-Newton descent direction
    descent_dir = conjugate_gradient(sc.values(), loss, sc)

    # Optimizer: take a Gauss Newton descent step
    sc.step(descent_dir)

    # Compute the loss using the current parameter values
    image, loss = sc.eval_func()

    for k in key: 
        print(f"{RED}>>> Iteration {it:02d}: loss = {loss} \n\t\t  params[key] = {params[k]}{RESET}\n", end='\n')
print('\nOptimization complete.')

print(f"\n\n{sc.expected_value()}")

for k in key:
    print(f"  Get: {params[k]}")


3


TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. drjit.cuda.ad.TensorXf64()
    2. drjit.cuda.ad.TensorXf64(array: object)
    3. drjit.cuda.ad.TensorXf64(array: drjit.cuda.ad.Float64)
    4. drjit.cuda.ad.TensorXf64(array: drjit.cuda.ad.Float64, shape: List[int])
    5. drjit.cuda.ad.TensorXf64(arg0: drjit.cuda.ad.TensorXi)
    6. drjit.cuda.ad.TensorXf64(arg0: drjit.cuda.ad.TensorXu)
    7. drjit.cuda.ad.TensorXf64(arg0: drjit.cuda.ad.TensorXi64)
    8. drjit.cuda.ad.TensorXf64(arg0: drjit.cuda.ad.TensorXu64)
    9. drjit.cuda.ad.TensorXf64(arg0: drjit.cuda.ad.TensorXf)
    10. drjit.cuda.ad.TensorXf64(arg0: drjit.cuda.ad.TensorXf64)
    11. drjit.cuda.ad.TensorXf64(arg0: drjit.cuda.TensorXf64)

Invoked with: None

In [ ]:
# low resolution